In [1]:
import math
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import re as re
% matplotlib inline
import Getmissed
import Processothers
import Addmissingvalue
from sklearn.linear_model import LogisticRegression
import Addpredict
import Predict
from sklearn.linear_model import Ridge
from imblearn.over_sampling import SMOTE
import Predict_Estimation
from sklearn.model_selection import cross_val_score,train_test_split
import Predict_profit
from sklearn.metrics import r2_score
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingClassifier
import seaborn as sns

#read data
train_df = pd.read_csv('DataTraining.csv')
test_df = pd.read_csv('DataPredict.csv')
#fill missing data
train=Getmissed.GetTrain(train_df)
train_df=Processothers.Get_processed_and_complete(train_df,train)
#get data for profit
[X_profit,y_profit]=Processothers.Get_processed_profit_need_data(train_df)
#SMOTE
train_df=Processothers.Process_SMOTE(train_df)
#get data for responded
[X_all,y_all]=Processothers.Process_get_Xdata_y(train_df,0)
#fill missing data for test dataset
test_df=Processothers.Get_processed_and_complete(test_df,train)
test_df=Processothers.Process_get_nice_test_df(test_df,train_df)
#normalise
test_df=Predict_profit.judge_process(test_df)
X_all=Predict_profit.judge_process(X_all)
X_profit=Predict_profit.judge_process(X_profit)
#get final CSV and totalprofit
#test_df_GBDT_good,totalptofit_GBDT=Predict.Final_predict(test_df,X_all,y_all,X_profit,y_profit,1,0.9476)
#test_df_GBDT_good.to_csv("DataPredict.csv",index=False,sep=',')

/home/chenyf1/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
X_train,X_test,y_train,y_test=train_test_split(X_all,y_all,test_size=0.2,random_state=2)

#Svm_score = cross_val_score(Svm, X_all, y_all, cv=10, scoring='accuracy')
#print 'The accuracy by using SVM is:', Svm_score.mean().round(4)


In [3]:
import time

time_start=time.time()

from sklearn import svm
clf = svm.SVC(C=0.1,kernel='linear')
clf.fit(X_train, y_train)
#Svm_score = cross_val_scor
print clf.score(X_test,y_test)
#print clf.score(X_train,y_train)
time_end=time.time()
print('totally cost',time_end-time_start)

0.8142954856361149
('totally cost', 8.684634923934937)


In [4]:
M=clf.n_support_

In [5]:
M=M[0]+M[1]

In [6]:
n=X_train.shape[0]

In [7]:
ln_n=math.log(n)

In [8]:
alpha=0.1
m=int((alpha*M*ln_n).round(0))

In [9]:
m

5845

In [10]:
import time

time_start=time.time()
S=np.random.normal(0,1,(65,65))
time_end=time.time()
print('totally cost',time_end-time_start)

('totally cost', 0.00039505958557128906)


In [11]:
S.shape

(65, 65)

In [12]:
S.shape

(65, 65)

In [13]:
max(1,2)

2

In [14]:
clf.dual_coef_.shape

(1, 6240)

In [15]:
X_train.shape

(11695, 65)

In [16]:
M

6240

In [17]:
w=np.dot(X_train.values,clf.coef_.T)
b=clf.intercept_

In [18]:
sum=0
for i in range(0,w.size):
    sum=sum+max(0,(1-(y_train.values[i]*(w[i]+b))))

In [19]:
SX=np.dot(S,X_train.values)

/home/chenyf1/anaconda2/lib/python2.7/site-packages/_pytest/fixtures.py:844: DeprecationWarning: The `convert` argument is deprecated in favor of `converter`.  It will be removed after 2019/01.
  params = attr.ib(convert=attr.converters.optional(tuple))
/home/chenyf1/anaconda2/lib/python2.7/site-packages/_pytest/fixtures.py:846: DeprecationWarning: The `convert` argument is deprecated in favor of `converter`.  It will be removed after 2019/01.
  ids = attr.ib(default=None, convert=_ensure_immutable_ids)


ValueError: shapes (65,65) and (11695,65) not aligned: 65 (dim 1) != 11695 (dim 0)

In [ ]:
Sy=np.dot(S,y_train.values)

In [ ]:
import time

time_start=time.time()

from sklearn import svm
clf_s = svm.SVC(C=0.1,kernel='linear')
clf_s.fit(SX, Sy)
#Svm_score = cross_val_scor
#print clf.score(X_test,y_test)
#print clf.score(X_train,y_train)
time_end=time.time()
print('totally cost',time_end-time_start)

In [ ]:
Sy

In [ ]:
y_train.values

In [ ]:
f1=np.linalg.norm(clf.coef_)**2+sum

In [ ]:
f1

In [ ]:
X_s=np.dot(S,X_train)
y_s=np.dot(S,y_train)

In [ ]:
from cvxpy import *

In [24]:
from cvxpy import *
import numpy
import mosek
import matplotlib.pyplot as plt

# Problem data.
n = X_train.shape[0]
m = X_train.shape[1]
#numpy.random.seed(1)
A = X_train.T.values
y = y_train.values
D=np.diag(y)
B=np.dot(A,D)
# gamma must be positive due to DCP rules.
#gamma = Parameter(sign="positive")

# Construct the problem.
x = Variable(n)
obj = Minimize(sum_squares(B*x))
constraints = [x>=0, sum(x)==1]
prob = Problem(obj,constraints)
prob.solve()

2.407481330989855e-31

In [ ]:
A = X_train.T.values
y = y_train.values
D=np.diag(y)
B=np.dot(A,D)

In [26]:
DataFrame(x.value)

,0
0,-6.372346e-20
1,1.707359e-04
2,-1.153600e-20
3,-7.830034e-20
4,-1.800362e-20
5,1.707359e-04
6,1.707359e-04
7,-2.462730e-20
8,1.707359e-04
9,1.707359e-04


In [ ]:
np.sum(x.value)

In [ ]:
prob.status

In [ ]:
np.sum(x.value<0)

In [ ]:
np.sum(x.value<0)

In [ ]:
prob.value

In [ ]:
f1=np.linalg.norm(np.dot(B,x.value))

In [ ]:
np.dot(B,x.value).shape

In [ ]:
f1

In [ ]:
from cvxpy import *
import numpy
import matplotlib.pyplot as plt

# Problem data.
n = X_train.shape[0]
m = X_train.shape[1]
#numpy.random.seed(1)
A = X_train.T.values
y = y_train.values
D=np.diag(y)
B=np.dot(A,D)
# gamma must be positive due to DCP rules.
#gamma = Parameter(sign="positive")
SB=np.dot(S,B)
# Construct the problem.
x = Variable(n)
obj = Minimize(sum_squares(SB*x))
constraints = [x>=0, sum(x)==1]
prob = Problem(obj,constraints)
prob.solve(solver=SCS)

In [ ]:
np.sum(x.value<0)

In [ ]:
prob.value

In [ ]:
m


In [ ]:
n

In [ ]:
B.shape

In [ ]:
f2=np.linalg.norm(np.dot(B,x.value))

In [ ]:
f2

In [ ]:
f1

In [ ]:
f2/f1

In [ ]:
SB.shape

In [ ]:
prob.value

In [ ]:
from cvxpy import *
import numpy
import matplotlib.pyplot as plt


x = Variable()
obj = Minimize(sum_squares(x))
constraints = [x>=1]
prob = Problem(obj,constraints)
prob.solve()